<a href="https://colab.research.google.com/github/AhmedMM1122/Insect-task/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import Libraries
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import zscore
import time
import scipy.special
from scipy.special import gamma, gammaln
from scipy.stats import mode
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.decomposition import PCA
import scipy.io
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.linalg import block_diag
import argparse

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#load dataset
dataset = '/content/drive/MyDrive/data/INSECTS' #change this donia

fname1 = f'/content/drive/MyDrive/data/INSECTS/data.mat' #change this donia
fname2 = f'/content/drive/MyDrive/data/INSECTS/splits.mat'#change this donia

data = scipy.io.loadmat(fname1)
splits = scipy.io.loadmat(fname2)

In [8]:
embeddings_dna = data['embeddings_dna']
embeddings_img = data['embeddings_img']
labels = data['labels']
trainval_loc = splits['trainval_loc']
train_loc = splits['train_loc']
test_seen_loc = splits['test_seen_loc']
test_unseen_loc = splits['test_unseen_loc']
val_seen_loc = splits['val_seen_loc']
val_unseen_loc = splits['val_unseen_loc']
G=data['G']

In [9]:
model = 'OSBC_DNA'
transductive = True

In [10]:
#Ridge_regression

def ridge_regression(embeddings_dna, embeddings_img, data_subset, lambda_,va_l1,va_l2):
    di = embeddings_img.shape[1]
    st = data_subset
    X = embeddings_img[st[va_l1], :].T
    D = embeddings_dna[st[va_l2], :].T

    DXT = np.dot(D, X.T)
    V = np.dot(DXT, np.linalg.inv(np.dot(X, X.T) + rho * np.eye(di)))

    return V

In [11]:
if transductive:
  dd = embeddings_dna.shape[1]
  di = embeddings_img.shape[1]
  rho = 1
  if model == 'OSBC_IMG':
    print("Transductive model works well if the mapping is from Image to DNA,")
    print("Thus we automatically run this version!")
  model = 'OSBC_DIT'
  st = np.concatenate((trainval_loc, test_unseen_loc, test_seen_loc), axis=1)
  embeddings_dna = zscore(embeddings_dna, axis=0)
  embeddings_img = zscore(embeddings_img, axis=0)
  st = np.array(st).flatten()
  valid_indices = (st >= 0) & (st < embeddings_img.shape[0])
  valid_indices1 = (st >= 0) & (st < embeddings_dna.shape[0])
  start_time = time.time()
  V = ridge_regression(embeddings_dna, embeddings_img, st, rho,valid_indices,valid_indices1)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print('Time took for learning map in transductive setup:', elapsed_time)

Time took for learning map in transductive setup: 8.823112726211548


In [12]:
pca_dim = 500

In [17]:
def load_tuned_params(model):
 #We use the upper() method to convert the input model string to uppercase
    model = model.upper()
 #We define the parameter sets OSBC_IMG, OSBC_DNA, OSBC_DIL, and OSBC_DIT as lists.
    OSBC_IMG = [0.1, 10, 5 * pca_dim, 1]
    OSBC_DNA = [0.1, 10, 25 * pca_dim, 0.5]
    OSBC_DIL = [0.1, 10, 5 * pca_dim, 0.5]
    OSBC_DIT = [0.1, 10, 25 * pca_dim, 0.5]
#evluate model of data, if it is img,dna,.......
    if model == 'OSBC_IMG':
        data = OSBC_IMG
    elif model == 'OSBC_DNA':
        data = OSBC_DNA
    elif model == 'OSBC_DIL':
        data = OSBC_DIL
    elif model == 'OSBC_DIT':
        data = OSBC_DIT
    else:
        raise ValueError("Invalid model name")
#The selected parameters are unpacked into k_0, k_1, m, and s, and these values are returned as a tuple
    k_0, k_1, m, s = data
    return k_0, k_1, m, s,pca_dim


In [27]:
load_tuned_params(model, k_0, k_1, m, s, pca_dim)

NameError: ignored